In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from itertools import combinations
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
from google.colab import drive

drive.mount('/content/drive')

rankings = pd.read_csv('/content/drive/MyDrive/soccer_data/test_data/fifa_ranking-2022-08-25.csv')
rankings = rankings.loc[:,['rank', 'country_full', 'country_abrv', 'cur_year_avg_weighted', 'rank_date', 
                           'two_year_ago_weighted', 'three_year_ago_weighted']]
rankings = rankings.replace({"IR Iran": "Iran"})
rankings['weighted_points'] =  rankings['cur_year_avg_weighted'] + rankings['two_year_ago_weighted'] + rankings['three_year_ago_weighted']
rankings['rank_date'] = pd.to_datetime(rankings['rank_date'])

matches = pd.read_csv('/content/drive/MyDrive/soccer_data/test_data/results.csv', encoding='cp949')
matches =  matches.replace({'Germany DR': 'Germany', 'China': 'China PR', 'United States':'USA',"South Korea" : "Korea Republic"})
matches['date'] = pd.to_datetime(matches['date'])

world_cup = pd.read_csv('/content/drive/MyDrive/soccer_data/test_data/World Cup 2022 Dataset.csv')
world_cup = world_cup.loc[:, ['Team', 'Group', 'First match against', 'Second match against', 'Third match against']]
world_cup = world_cup.dropna(how='all')
world_cup = world_cup.replace({"IRAN": "Iran", 
                               "Costarica": "Costa Rica", 
                               "Korea" : "Korea Republic"})
world_cup = world_cup.set_index('Team')


# I want to have the ranks for every day 
rankings = rankings.set_index(['rank_date'])\
            .groupby(['country_full'], group_keys=False)\
            .resample('D').first()\
            .fillna(method='ffill')\
            .reset_index()

# join the ranks
matches = matches.merge(rankings, 
                        left_on=['date', 'home_team'], 
                        right_on=['rank_date', 'country_full'])
matches = matches.merge(rankings, 
                        left_on=['date', 'away_team'], 
                        right_on=['rank_date', 'country_full'], 
                        suffixes=('_home', '_away'))
# feature generation
matches['rank_difference'] = matches['rank_home'] - matches['rank_away']
matches['average_rank'] = (matches['rank_home'] + matches['rank_away'])/2
matches['point_difference'] = matches['weighted_points_home'] - matches['weighted_points_away']
matches['score_difference'] = matches['home_score'] - matches['away_score']
matches['is_won'] = matches['score_difference'] > 0 # take draw as lost
matches['is_stake'] = matches['tournament'] != 'Friendly'
matches['giant_kill'] = matches['rank_difference'] * matches['score_difference']


# I tried earlier rest days but it did not turn to be useful
# min_rest = 0
# max_rest = 30
#print(matches['date'].dt.day.astype(int));

#print(matches.groupby('home_team').diff(matches['date'].dt.day))
#matches['rest_days'] = matches.groupby('home_team').diff()['date'].dt.day.astype(int).clip(min_rest,max_rest).fillna(max_rest)



# I tried earlier the team as well but that did not make a difference either
#matches['wc_participant'] = matches['home_team'] * matches['away_team'].isin(world_cup.index.tolist())
matches['wc_participant'] = matches['home_team'] * matches['home_team'].isin(world_cup.index.tolist())
matches['wc_participant'] = matches['wc_participant'].replace({'':'Other'})
matches = matches.join(pd.get_dummies(matches['wc_participant']))


#테스트셋 확인
X, y = matches.loc[:,['average_rank', 'rank_difference', 'point_difference', 'is_stake']], matches['is_won']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

logreg = linear_model.LogisticRegression(C=1e-5)
features = PolynomialFeatures(degree=2)
model = Pipeline([
    ('polynomial_features', features),
    ('logistic_regression', logreg)
])
model = model.fit(X_train, y_train)


# #figures 
# fpr, tpr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
# plt.figure(figsize=(15,5))
# ax = plt.subplot(1,3,1)
# ax.plot([0, 1], [0, 1], 'k--')
# ax.plot(fpr, tpr)
# ax.set_title('AUC score is {0:0.2}'.format(roc_auc_score(y_test, model.predict_proba(X_test)[:,1])))
# ax.set_aspect(1)

# ax = plt.subplot(1,3,2)
# cm = confusion_matrix(y_test, model.predict(X_test))
# ax.imshow(cm, cmap='Blues', clim = (0, cm.max())) 

# ax.set_xlabel('Predicted label')
# ax.set_title('Performance on the Test set')

# ax = plt.subplot(1,3,3)
# cm = confusion_matrix(y_train, model.predict(X_train))
# ax.imshow(cm, cmap='Blues', clim = (0, cm.max())) 
# ax.set_xlabel('Predicted label')
# ax.set_title('Performance on the Training set')
# pass

# #선형그래프
# features = ['average_rank', 'rank_difference', 'point_difference']
# wrongs = y_test != model.predict(X_test)

# for feature in features:
#     plt.figure()
#     plt.title(feature)
#     X_test.loc[wrongs, feature].plot.kde()
#     X.loc[:, feature].plot.kde()
#     plt.legend(['wrongs', 'all'])
    
# print("Stakes distribution in the wrong predictions")
# print(X_test.loc[wrongs, 'is_stake'].value_counts() / wrongs.sum())
# print("Stakes distribution overall")
# print(X['is_stake'].value_counts() / X.shape[0])

# # let's define a small margin when we safer to predict draw then win


margin = 0.05

# let's define the rankings at the time of the World Cup
world_cup_rankings = rankings.loc[(rankings['rank_date'] == rankings['rank_date'].max()) & 
                                    rankings['country_full'].isin(world_cup.index.unique())]
# print(world_cup_rankings,'\n')
world_cup_rankings = world_cup_rankings.set_index(['country_full'])

opponents = ['First match against', 'Second match against', 'Third match against']

world_cup['points'] = 0
world_cup['total_prob'] = 0

arr_country = ['Qatar','Ecuador','Senegal','Netherlands','England','Iran','USA','Wales','Argentina','Saudi Arabia','Mexico','Poland','France','Australia','Denmark','Tunisia','Spain','Costa Rica','Germany','Japan','Belgium','Canada','Morocco','Croatia','Brazil','Serbia','Switzerland','Cameroon','Portugal','Ghana','Uruguay','Korea Republic']

arr_predict = [[[[0 for l in range(4)] for k in range(8)]for j in range(4)]for i in range(8)]
# print(world_cup)
for i in range(32):
  home = arr_country[i]
  for j in range(32):
    away = arr_country[j]
    #print("{} vs. {}: ".format(home, away), end='')
    row = pd.DataFrame(np.array([[np.nan, np.nan, np.nan, True]]), columns=X_test.columns)
    #print(world_cup_rankings.loc[home,])
    home_rank = world_cup_rankings.loc[home, 'rank']
    home_points = world_cup_rankings.loc[home, 'weighted_points']
    opp_rank = world_cup_rankings.loc[away, 'rank']
    opp_points = world_cup_rankings.loc[away, 'weighted_points']
    row['average_rank'] = (home_rank + opp_rank) / 2
    row['rank_difference'] = home_rank - opp_rank
    row['point_difference'] = home_points - opp_points
    home_win_prob = model.predict_proba(row)[:,1][0]
    #print(str(int(i%8)*4 + int(i/8))+" vs. "+str(int(j%8) * 4 + int(j/8))+" / "+str(home_win_prob))

    arr_predict[int(i/4)][int(i%4)][int(j/4)][int(j%4)] = home_win_prob
  #print(' ')
    # world_cup.loc[home, 'total_prob'] += home_win_prob
    # world_cup.loc[away, 'total_prob'] += 1-home_win_prob
    
    # # if (home_rank - opp_rank) > 0 :
    # #   row['giant_kill'] = home_points - opp_points
    # points = 0
    # if home_win_prob <= 0.5 - margin:
    #     print("{} wins with {:.2f}".format(away, 1-home_win_prob))
    #     world_cup.loc[away, 'points'] += 3
    # if home_win_prob > 0.5 - margin:
    #     points = 1
    # if home_win_prob >= 0.5 + margin:
    #     points = 3
    #     world_cup.loc[home, 'points'] += 3
    #     print("{} wins with {:.2f}".format(home, home_win_prob))
    # if points == 1:
    #     print("Draw")
    #     world_cup.loc[home, 'points'] += 1
    #     world_cup.loc[away, 'points'] += 1





print(' ')
tmp_predict = [[[[0 for l in range(4)] for k in range(8)]for j in range(4)]for i in range(8)]
for hteam in range(32):
  for ateam in range(32):
    if(hteam == ateam): #미러전이면 0.5
      tmp_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)]= 0.5
    elif(hteam == 0): #홈이 카타르인 경우 그대로
      tmp_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)] =  arr_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)]
    elif(ateam == 0): #어웨이가 카타르인 경우 홈이 카타르인 확률로
      tmp_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)]= (1-arr_predict[int(ateam/4)][int(ateam%4)][int(hteam/4)][int(hteam%4)])    
    ## 이란 사우디 홈팀 승률로 변경
    else:#기타는 홈어웨이 평균치로
      tmp_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)] = (arr_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)] + (1-arr_predict[int(ateam/4)][int(ateam%4)][int(hteam/4)][int(hteam%4)])) / 2

for hteam in range(32):
  for ateam in range(32):   
    print(arr_country[hteam]+str(hteam)+" vs. "+arr_country[ateam]+str(ateam)+" / new: "+str(tmp_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)])+" / old home: "+str(arr_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)]) + " / old away: "+str(1-arr_predict[int(ateam%8)][int(ateam/8)][int(hteam%8)][int(hteam/8)])+' /  home wei'+str(world_cup_rankings.loc[arr_country[hteam], 'weighted_points'])+' / away wei: '+str(world_cup_rankings.loc[arr_country[ateam], 'weighted_points']))
    arr_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)] = tmp_predict[int(hteam/4)][int(hteam%4)][int(ateam/4)][int(ateam%4)]
  print(' ')


# arr = [[0 for j in range(cols)] for i in range(rows)]
arr_count_fs = [[0 for j in range(2)] for i in range(32)]
arr_groupwin = [[0 for j in range(2)] for i in range(32)]
for group_num in range(8):
  #??
  for round1 in range(2): # 그룹 내 1,2번 매치
    for round2 in range(2): # 그룹 내 1,3번 매치
      for round3 in range(2): # 그룹 내 1,4번 매치
        for round4 in range(2): # 그룹 내 2,3번 매치
          for round5 in range(2): # 그룹 내 2,4번 매치
            for round6 in range(2): # 그룹 내 3,4번 매치

              group_point = [0,0,0,0]
              this_round = 32*(round1) + 16*(round2) + 8*(round3) + 4*(round4) + 2*(round5) + 1*(round6)
              score_arr = [0,0,0,0]
              predict_arr = [0,0,0,0,0,0]

              if round1 == 0:
                predict_arr[0] += arr_predict[group_num][0][group_num][1]
                group_point[0] += 3
              elif round1 == 1 :
                predict_arr[0] += (1-arr_predict[group_num][0][group_num][1])
                group_point[1] += 3

              if round2 == 0:
                predict_arr[1] += arr_predict[group_num][0][group_num][2]
                group_point[0] += 3
              elif round2 == 1 :
                predict_arr[1] += (1-arr_predict[group_num][0][group_num][2])
                group_point[2] += 3

              if round3 == 0:
                predict_arr[2] += arr_predict[group_num][0][group_num][3]
                group_point[0] += 3
              elif round3 == 1 :
                predict_arr[2] += (1-arr_predict[group_num][0][group_num][3])
                group_point[3] += 3

              if round4 == 0:
                predict_arr[3] += arr_predict[group_num][1][group_num][2]
                group_point[1] += 3
              elif round4 == 1 :
                predict_arr[3] += (1-arr_predict[group_num][1][group_num][2])
                group_point[2] += 3

              if round5 == 0:
                predict_arr[4] += arr_predict[group_num][1][group_num][3]
                group_point[1] += 3
              elif round5 == 1 :
                predict_arr[4] += (1-arr_predict[group_num][1][group_num][3])
                group_point[3] += 3

              if round6 == 0:
                predict_arr[5] += arr_predict[group_num][2][group_num][3]
                group_point[2] += 3
              elif round6 == 1 :
                predict_arr[5] += (1-arr_predict[group_num][2][group_num][3])
                group_point[3] += 3
              
              
              #여기서 1,2등 체크하고 넘어가기?
              gfchk = [0,0,0,0] # 그룹 내 어떤 팀이 1등인지 확인
              gschk = [0,0,0,0]
              gfcnt = 0 #기존에 1등이 있는지 확인
              gscnt = 0

              gfirst = -1 # 1등의 승점
              gsecond = -1
              for cal in range(4):
                if(gfirst < group_point[cal]): # 기존 1등보다 승점이 높으면

                  if(gfcnt > 0): # 기존에 1등이 있었으면
                    gftmp = [0 for i in range(4)] #임시 저장 on

                    if(gscnt > 0): #기존에 2등이 있었으면
                      for gscle in range(4):
                        gschk[gscle] = 0 # 2등 로그 초기화
                        if(gfchk[gscle] == 1): #루프 돌리는 김에 기존 1등 몇팀인지 탐색
                          gschk[gscle] = 1 # 1등으로 바꾸기
                      gscnt = gfcnt # 1등팀 수를 2등팀수에 갈아끼우기
                      gsecond = gfirst # 기존 승점 갈아끼우기

                    elif(gscnt == 0): #기존에 2등팀이 없었으면
                      for gscle in range(4):
                        if(gfchk[gscle] == 1): #기존 1등인걸 찾으면
                          gschk[gscle] = 1
                      gscnt = gfcnt # 1등팀 수를 2등팀수에 갈아끼우기
                      gsecond = gfirst # 기존 승점 갈아끼우기

                    for gfcle in range(4): # 기존 1등 로그 지우기
                      gfchk[gfcle] = 0

                  #기존 1등 유무와 상관없는 공통 수행
                  #1등 승점 변경 / 1등 변경 / 1등 수 1명으로 변경
                  gfirst = group_point[cal]
                  gfchk[cal] = 1
                  gfcnt = 1

                elif(gfirst == group_point[cal]): #기존 1등이랑 점수가 같으면
                  gfchk[cal] = 1
                  gfcnt += 1

                else: # 1등보다 점수가 작은 경우
                  #label3
                  if(gsecond < group_point[cal]): # 기존 2등보다는 승점이 높을 때
                    if(gscnt > 0): #기존에 2등이 있었다면
                      for gscle in range(4): # 초기화하고
                        gschk[gscle] = 0
                    gsecond = group_point[cal] # 바꾸기
                    gschk[cal] = 1
                    gscnt = 1
                  elif(gsecond == group_point[cal]): #기존 2등과 승점이 같을 때
                    gschk[cal] = 1
                    gscnt += 1
                  

              #print('매치번호: '+str(group_num+1) + ' ' + str(this_round))
              
              # if(gfcnt == 1): # 단독 1등인 경우
              #   for fnd in range(4):
              #     if(gfchk[fnd]==1): # 어떤 팀이 1등인지 찾아서 승률 저장
              #       arr_groupwin[group_num*4+fnd][0] += predict_arr[0]*predict_arr[1]*predict_arr[2]*predict_arr[3]*predict_arr[4]*predict_arr[5]
              #       #print('1등: '+arr_country[group_num*4+rec_cnt])

              # elif(gfcnt > 1): #1등 동일승점이 많은 경우
              #   rec = [0 for j in range(gfcnt)] # 공동 1등 수 만큼 임시변수 생성
              #   rec_pred = [0 for j in range(gfcnt)]
              #   rec_wei = [0 for j in range(gfcnt)]
              #   rec_chk = 0
              #   sum_wei = 0
              #   for fnd in range(4): 
              #     if(gfchk[fnd]==1): # 1등인 국가 탐색
              #       rec[rec_chk] = fnd # 1등인 국가 저장
              #       rec_pred[rec_chk] = predict_arr[0]*predict_arr[1]*predict_arr[2]*predict_arr[3]*predict_arr[4]*predict_arr[5] # 해당 경우의수 확률 저장
              #       rec_wei[rec_chk] = world_cup_rankings.loc[arr_country[fnd], 'weighted_points'] # 가중치 불러오기
              #       sum_wei += world_cup_rankings.loc[arr_country[fnd], 'weighted_points'] # 가중치 합계 연산
              #       rec_chk += 1 # 체크용 변수 ++
              #   for rec_cnt in range(gfcnt):
              #     arr_groupwin[group_num*4+rec[rec_cnt]][0] += rec_pred[rec_cnt]*(rec_wei[rec_cnt]/sum_wei) # 1등인 확률 = 해당 경우의수 확률 * (동일 등수 가중치 비례)
              #     #print('1등: '+arr_country[group_num*4+rec_cnt])

              
              # if(gscnt == 1): # 2등이 한 팀인 경우
              #   for fnd in range(4):
              #     if(gschk[fnd]==1):
              #       arr_groupwin[group_num*4+fnd][1] += predict_arr[0]*predict_arr[1]*predict_arr[2]*predict_arr[3]*predict_arr[4]*predict_arr[5]
              #       #print('2등: '+arr_country[group_num*4+rec_cnt])

              # elif(gscnt > 1):
              #   rec = [0 for j in range(gscnt)]
              #   rec_pred = [0 for j in range(gscnt)]
              #   rec_wei = [0 for j in range(gscnt)]
              #   rec_chk = 0
              #   sum_wei = 0
              #   for fnd in range(4):
              #     if(gschk[fnd]==1):
              #       rec[rec_chk] = fnd
              #       rec_pred[rec_chk] = predict_arr[0]*predict_arr[1]*predict_arr[2]*predict_arr[3]*predict_arr[4]*predict_arr[5]
              #       rec_wei[rec_chk] = world_cup_rankings.loc[arr_country[fnd], 'weighted_points']
              #       sum_wei += world_cup_rankings.loc[arr_country[fnd], 'weighted_points']
              #       rec_chk += 1
              #   for rec_cnt in range(gscnt):
              #     arr_groupwin[group_num*4+rec[rec_cnt]][1] += rec_pred[rec_cnt]*(rec_wei[rec_cnt]/sum_wei)
              #     #print('2등: '+arr_country[group_num*4+rec_cnt])\
              # else:
              #   print('매치번호: '+str(group_num) + ' ' + str(round1)+str(round2)+str(round3)+str(round4)+str(round5)+str(round6)+" error "+str(group_point[0])+' '+str(group_point[1])+' '+str(group_point[2])+' '+str(group_point[3])+' ')

              rec = [0 for j in range(gfcnt)] # 1등 수 만큼 임시변수 생성
              rec_wei = [0 for j in range(gfcnt)]
              rec_chk = 0
              sum_wei = 0
              rec_pred = predict_arr[0]*predict_arr[1]*predict_arr[2]*predict_arr[3]*predict_arr[4]*predict_arr[5]
              for fnd in range(4): 
                if(gfchk[fnd]==1): # 1등인 국가 탐색
                  rec[rec_chk] = fnd # 1등인 국가 저장
                  sum_wei += world_cup_rankings.loc[arr_country[fnd], 'weighted_points'] # 가중치 합계 연산
                  rec_chk += 1 # 체크용 변수 ++
              # for rec_cnt in range(gfcnt):
              #   arr_groupwin[group_num*4+rec[rec_cnt]][0] += rec_pred*(world_cup_rankings.loc[arr_country[rec[rec_cnt]], 'weighted_points']/sum_wei) # 1등인 확률 = 해당 경우의수 확률 * (동일 등수 가중치 비례)
              #   #print(chr(group_num+65)+'1등: '+arr_country[group_num*4+rec[rec_cnt]]+str(arr_groupwin[group_num*4+rec[rec_cnt]][0]))
              #   arr_count_fs[group_num*4+rec[rec_cnt]][0] += 1
              #   print(group_num*4+rec[rec_cnt])
              for fnd in range(4):
                if(gfchk[fnd]==1): # 1등인 국가 탐색
                  arr_groupwin[group_num*4+fnd][0] += rec_pred*(world_cup_rankings.loc[arr_country[fnd], 'weighted_points']/sum_wei) # 1등인 확률 = 해당 경우의수 확률 * (동일 등수 가중치 비례)
                  #print(chr(group_num+65)+'1등: '+arr_country[group_num*4+rec[rec_cnt]]+str(arr_groupwin[group_num*4+rec[rec_cnt]][0]))
                  arr_count_fs[group_num*4+fnd][0] += 1

            

              rec = [0 for j in range(gscnt)]
              rec_wei = [0 for j in range(gscnt)]
              rec_chk = 0
              sum_wei = 0
              rec_pred = predict_arr[0]*predict_arr[1]*predict_arr[2]*predict_arr[3]*predict_arr[4]*predict_arr[5]
              for fnd in range(4):
                if(gschk[fnd]==1):
                  rec[rec_chk] = fnd
                  sum_wei += world_cup_rankings.loc[arr_country[fnd], 'weighted_points']
                  rec_chk += 1
              # for rec_cnt in range(gscnt):
              #   arr_groupwin[group_num*4+rec[rec_cnt]][1] += rec_pred*(world_cup_rankings.loc[arr_country[rec[rec_cnt]], 'weighted_points']/sum_wei)
              #   #print(chr(group_num+65)+'2등: '+arr_country[group_num*4+rec[rec_cnt]]+str(arr_groupwin[group_num*4+rec[rec_cnt]][1]))
              #   arr_count_fs[group_num*4+rec[rec_cnt]][1] += 1
              #   #print(group_num*4+rec[rec_cnt])
              for fnd in range(4):
                if(gschk[fnd]==1): # 1등인 국가 탐색
                  arr_groupwin[group_num*4+fnd][1] += rec_pred*(world_cup_rankings.loc[arr_country[fnd], 'weighted_points']/sum_wei) # 1등인 확률 = 해당 경우의수 확률 * (동일 등수 가중치 비례)
                  #print(chr(group_num+65)+'1등: '+arr_country[group_num*4+rec[rec_cnt]]+str(arr_groupwin[group_num*4+rec[rec_cnt]][0]))
                  arr_count_fs[group_num*4+fnd][1] += 1

# for i in range(32):
#   for j in range(2):
#     print(chr(i+65),' ',j+1,'등 : ',arr_country[i],' / ',arr_count_fs[i][j])

# sum_los = 0 #탈락확률 누계

print('')
sum_fir = 0;
sum_sec = 0;
for i in range(32):
  print(arr_country[i],'\t\t 조 1위 확률 :\t',arr_groupwin[i][0],'\t조 2위 확률 :\t',arr_groupwin[i][1])
  sum_fir += arr_groupwin[i][0]
  sum_sec += arr_groupwin[i][1]
  if((i+1)%4 == 0):
    print('---------------------------------------------------')
    print(chr(int((i+1)/4)+64),'조 1위 확률 :\t',sum_fir,'\t조 2위 확률 :\t',sum_sec)
    print(' ')
    sum_fir = 0
    sum_sec = 0


#좌 우 분할
#좌 : A0 B1 C0 D1 E0 F1 G0 H1
#우 : A1 B0 C1 D0 E1 F0 G1 H0

# 그룹 넘버 % 2 == 0 이면 조별 0
# 그룹 넘버 % 2 != 0 이면 조별 1

arr_round16 = [[[0 for k in range(2)] for j in range(2)] for i in range(32)] #16라운드 팀별 승패 확률
# arr_round16[team_num][left_right][win_lose]

for cut_match in range(8): # AB / CD / EF / GH 컷
  for cut_groupnum in range(4): # 0123 / 4567 컷
    #print(str(cut_match*4+cut_groupnum))
    for leri in range(2):
      if(cut_match%2 == 0): # A C E G
        for winlos in range(2):
          for opp in range(4):
            #print(str(int((cut_match*8+cut_groupnum)/4))+' '+str(int((cut_match*8+cut_groupnum)%4)) +' ' + str(cut_match*8+cut_groupnum) + '  vs  '+str(int(((cut_match+1)*8+opp)/4))+' '+ str(int(((cut_match+1)*8+opp)%4)) + ' '+str((cut_match+1)*8+opp))
            arr_round16[cut_match*4+cut_groupnum][winlos][0] += arr_groupwin[cut_match*4+cut_groupnum][winlos] * arr_groupwin[(cut_match+1)*4+opp][1-winlos] * arr_predict[cut_match][cut_groupnum][cut_match+1][opp] 
            arr_round16[cut_match*4+cut_groupnum][winlos][1] += arr_groupwin[cut_match*4+cut_groupnum][winlos] * arr_groupwin[(cut_match+1)*4+opp][1-winlos] * (1-arr_predict[cut_match][cut_groupnum][cut_match+1][opp])
            #print(str(cut_match*4+cut_groupnum)+' vs '+str((cut_match+1)*4+opp))
      else:
        for winlos in range(2):
          for opp in range(4):
            #print(str(int((cut_match*8+cut_groupnum)/4))+' '+str(int((cut_match*8+cut_groupnum)%4)) +' ' + str(cut_match*8+cut_groupnum) + '  vs  '+str(int(((cut_match+1)*8+opp)/4))+' '+ str(int(((cut_match+1)*8+opp)%4)) + ' '+str((cut_match+1)*8+opp))
            arr_round16[cut_match*4+cut_groupnum][winlos][0] += arr_groupwin[cut_match*4+cut_groupnum][winlos] * arr_groupwin[(cut_match-1)*4+opp][1-winlos] * arr_predict[cut_match][cut_groupnum][cut_match-1][opp]
            arr_round16[cut_match*4+cut_groupnum][winlos][1] += arr_groupwin[cut_match*4+cut_groupnum][winlos] * arr_groupwin[(cut_match-1)*4+opp][1-winlos] * (1-arr_predict[cut_match][cut_groupnum][cut_match-1][opp])
            #print(str(cut_match*4+cut_groupnum)+' vs '+str((cut_match-1)*4+opp))

sum_r16_winl = 0;
sum_r16_winr = 0;
sum_r16_losl = 0;
sum_r16_losr = 0;
for i in range(32):
  sum_r16_winl += arr_round16[i][0][0]
  sum_r16_winr += arr_round16[i][0][1]
  sum_r16_losl += arr_round16[i][1][0]
  sum_r16_losr += arr_round16[i][1][1]

sum_winl_dup = 0;
sum_winr_dup = 0;
sum_losl_dup = 0;
sum_losr_dup = 0;

for i in range(32):
  arr_round16[i][0][0] /= sum_r16_winl
  arr_round16[i][0][1] /= sum_r16_winr
  arr_round16[i][1][0] /= sum_r16_losl
  arr_round16[i][1][1] /= sum_r16_losr
  
  print(arr_country[i]+': 8강 진출확률L: '+str(arr_round16[i][0][0])+': 8강 진출확률R: '+str(arr_round16[i][1][0])+': 8강 진출확률avg: '+str((arr_round16[i][0][0]+arr_round16[i][1][0])/2)+': 16강 탈락 확률L: '+str(arr_round16[i][0][1])+': 16강 탈락 확률R: '+str(arr_round16[i][1][1])+': 16강 탈락 확률avg: '+str((arr_round16[i][0][1]+arr_round16[i][1][1])/2))
  sum_winl_dup += arr_round16[i][0][0]
  sum_winr_dup += arr_round16[i][0][1]
  sum_losl_dup += arr_round16[i][1][0]
  sum_losr_dup += arr_round16[i][1][1]
  if((i+1)%32 == 0):
    print('---------------------------------------------------')
    print('조 평균 8강 진출 확률L :\t',sum_winl_dup,'\t조 평균 8강 진출 확률R :\t',sum_winr_dup,'\t조 평균 8강 진출 확률avg :\t',(sum_winr_dup+sum_winl_dup)/2,'\t조 평균 16강 탈락 확률L :\t',sum_losl_dup,'\t조 평균 16강 탈락 확률R :\t',sum_losr_dup,'\t조 평균 16강 탈락 확률avg :\t',(sum_losr_dup+sum_losl_dup)/2)
    print(' ')
    sum_winl_dup = 0;
    sum_winr_dup = 0;
    sum_losl_dup = 0;
    sum_losr_dup = 0;

arr_round8 = [[[0 for k in range(2)] for j in range(2)] for i in range(32)] #8라운드 팀별 승패 확률

for cut_match in range(4): 
  for cut_groupnum in range(8):
    #print(str(cut_match*8+cut_groupnum))
    if(cut_match%2 == 0):
      for winlos in range(2):
        for opp in range(8):
          #print(str(int((cut_match*8+cut_groupnum)/4))+' '+str(int((cut_match*8+cut_groupnum)%4)) +' ' + str(cut_match*8+cut_groupnum) + '  vs  '+str(int(((cut_match+1)*8+opp)/4))+' '+ str(int(((cut_match+1)*8+opp)%4)) + ' '+str((cut_match+1)*8+opp)) 
          arr_round8[cut_match*8+cut_groupnum][winlos][0] += arr_round16[cut_match*8+cut_groupnum][winlos][0] * arr_round16[(cut_match+1)*8+opp][winlos][0] * arr_predict[int((cut_match*8+cut_groupnum)/4)][int((cut_match*8+cut_groupnum)%4)][int(((cut_match+1)*8+opp)/4)][int(((cut_match+1)*8+opp)%4)]
          arr_round8[cut_match*8+cut_groupnum][winlos][1] += arr_round16[cut_match*8+cut_groupnum][winlos][0] * arr_round16[(cut_match+1)*8+opp][winlos][0] * (1-arr_predict[int((cut_match*8+cut_groupnum)/4)][int((cut_match*8+cut_groupnum)%4)][int(((cut_match+1)*8+opp)/4)][int(((cut_match+1)*8+opp)%4)])
    else:
      for winlos in range(2):
        for opp in range(8):
          #print(str(int((cut_match*8+cut_groupnum)/4))+' '+str(int((cut_match*8+cut_groupnum)%4)) +' ' + str(cut_match*8+cut_groupnum) + '  vs  '+str(int(((cut_match-1)*8+opp)/4))+' '+ str(int(((cut_match-1)*8+opp)%4)) + ' '+str((cut_match-1)*8+opp))
          arr_round8[cut_match*8+cut_groupnum][winlos][0] += arr_round16[cut_match*8+cut_groupnum][winlos][0] * arr_round16[(cut_match-1)*8+opp][winlos][0] * arr_predict[int((cut_match*8+cut_groupnum)/4)][int((cut_match*8+cut_groupnum)%4)][int(((cut_match-1)*8+opp)/4)][int(((cut_match-1)*8+opp)%4)]
          arr_round8[cut_match*8+cut_groupnum][winlos][1] += arr_round16[cut_match*8+cut_groupnum][winlos][0] * arr_round16[(cut_match-1)*8+opp][winlos][0] * (1-arr_predict[int((cut_match*8+cut_groupnum)/4)][int((cut_match*8+cut_groupnum)%4)][int(((cut_match-1)*8+opp)/4)][int(((cut_match-1)*8+opp)%4)])



sum_r8_winl = 0;
sum_r8_winr = 0;
sum_r8_losl = 0;
sum_r8_losr = 0;
for i in range(32):
  sum_r8_winl += arr_round8[i][0][0]
  sum_r8_winr += arr_round8[i][0][1]
  sum_r8_losl += arr_round8[i][1][0]
  sum_r8_losr += arr_round8[i][1][1]

sum_winl_dup = 0;
sum_winr_dup = 0;
sum_losl_dup = 0;
sum_losr_dup = 0;

for i in range(32):
  arr_round8[i][0][0] /= sum_r8_winl
  arr_round8[i][0][1] /= sum_r8_winr
  arr_round8[i][1][0] /= sum_r8_losl
  arr_round8[i][1][1] /= sum_r8_losr
  
  print(arr_country[i]+': 4강 진출확률L: '+str(arr_round8[i][0][0])+': 4강 진출확률R: '+str(arr_round8[i][1][0])+': 4강 진출확률avg: '+str((arr_round8[i][0][0]+arr_round8[i][1][0])/2)+': 8강 탈락 확률L: '+str(arr_round8[i][0][1])+': 8강 탈락 확률R: '+str(arr_round8[i][1][1])+': 8강 탈락 확률avg: '+str((arr_round8[i][0][1]+arr_round8[i][1][1])/2))
  sum_winl_dup += arr_round8[i][0][0]
  sum_winr_dup += arr_round8[i][0][1]
  sum_losl_dup += arr_round8[i][1][0]
  sum_losr_dup += arr_round8[i][1][1]
  if((i+1)%32 == 0):
    print('---------------------------------------------------')
    print('조 평균 4강 진출 확률L :\t',sum_winl_dup,'\t조 평균 4강 진출 확률R :\t',sum_winr_dup,'\t조 평균 4강 진출 확률avg :\t',(sum_winr_dup+sum_winl_dup)/2,'\t조 평균 8강 탈락 확률L :\t',sum_losl_dup,'\t조 평균 8강 탈락 확률R :\t',sum_losr_dup,'\t조 평균 8강 탈락 확률avg :\t',(sum_losr_dup+sum_losl_dup)/2)
    print(' ')
    sum_winl_dup = 0;
    sum_winr_dup = 0;
    sum_losl_dup = 0;
    sum_losr_dup = 0;


arr_round4 = [[[0 for k in range(2)] for j in range(2)] for i in range(32)] #4라운드 팀별 승패 확률

for cut_match in range(2): 
  for cut_groupnum in range(16):
    #print(str(cut_match*16+cut_groupnum))
    if(cut_match%2 == 0): # A C E G
      for winlos in range(2):
        for opp in range(16):
          #print(str(int((cut_match*16+cut_groupnum)/4))+' '+str(int((cut_match*16+cut_groupnum)%4)) +' ' + str(cut_match*16+cut_groupnum) + '  vs  '+str(int(((cut_match+1)*16+opp)/4))+' '+ str(int(((cut_match+1)*16+opp)%4)) + ' '+str((cut_match+1)*16+opp)) 
          #print(str(int((cut_match*16+cut_groupnum)%8))+' '+str(int((cut_match*16+cut_groupnum)/8)) +' ' + str(cut_match*16+cut_groupnum) + '  vs  '+str(int(((cut_match+1)*16+opp)%8))+' '+ str(int(((cut_match+1)*16+opp)/8)) + ' '+str((cut_match+1)*16+opp)) 
          arr_round4[cut_match*16+cut_groupnum][winlos][0] += arr_round8[cut_match*16+cut_groupnum][winlos][0] * arr_round8[(cut_match+1)*16+opp][winlos][0] * arr_predict[int((cut_match*16+cut_groupnum)%8)][int((cut_match*16+cut_groupnum)/8)][int(((cut_match+1)*16+opp)%8)][int(((cut_match+1)*16+opp)/8)]
          arr_round4[cut_match*16+cut_groupnum][winlos][1] += arr_round8[cut_match*16+cut_groupnum][winlos][0] * arr_round8[(cut_match+1)*16+opp][winlos][0] * (1-arr_predict[int((cut_match*16+cut_groupnum)%8)][int((cut_match*16+cut_groupnum)/8)][int(((cut_match+1)*16+opp)%8)][int(((cut_match+1)*16+opp)/8)])
    else:
      for winlos in range(2):
        for opp in range(16):
          #print(str(int((cut_match*16+cut_groupnum)/4))+' '+str(int((cut_match*16+cut_groupnum)%4)) +' ' + str(cut_match*16+cut_groupnum) + '  vs  '+str(int(((cut_match-1)*16+opp)/4))+' '+ str(int(((cut_match-1)*16+opp)%4)) + ' '+str((cut_match-1)*16+opp))
          #print(str(int((cut_match*16+cut_groupnum)/4))+' '+str(int((cut_match*16+cut_groupnum)%4)) +' ' + str(cut_match*16+cut_groupnum) + '  vs  '+str(int(((cut_match-1)*16+opp)/4))+' '+ str(int(((cut_match-1)*16+opp)%4)) + ' '+str((cut_match-1)*16+opp))
          arr_round4[cut_match*16+cut_groupnum][winlos][0] += arr_round8[cut_match*16+cut_groupnum][winlos][0] * arr_round8[(cut_match-1)*16+opp][winlos][0] * arr_predict[int((cut_match*16+cut_groupnum)%8)][int((cut_match*16+cut_groupnum)/8)][int(((cut_match-1)*16+opp)%8)][int(((cut_match-1)*16+opp)/8)]
          arr_round4[cut_match*16+cut_groupnum][winlos][1] += arr_round8[cut_match*16+cut_groupnum][winlos][0] * arr_round8[(cut_match-1)*16+opp][winlos][0] * (1-arr_predict[int((cut_match*16+cut_groupnum)%8)][int((cut_match*16+cut_groupnum)/8)][int(((cut_match-1)*16+opp)%8)][int(((cut_match-1)*16+opp)/8)])


sum_r4_winl = 0;
sum_r4_winr = 0;
sum_r4_losl = 0;
sum_r4_losr = 0;
for i in range(32):
  sum_r4_winl += arr_round4[i][0][0]
  sum_r4_winr += arr_round4[i][0][1]
  sum_r4_losl += arr_round4[i][1][0]
  sum_r4_losr += arr_round4[i][1][1]

sum_winl_dup = 0;
sum_winr_dup = 0;
sum_losl_dup = 0;
sum_losr_dup = 0;

for i in range(32):
  arr_round4[i][0][0] /= sum_r4_winl
  arr_round4[i][0][1] /= sum_r4_winr
  arr_round4[i][1][0] /= sum_r4_losl
  arr_round4[i][1][1] /= sum_r4_losr
  print(arr_country[i]+': 결승 진출확률L: '+str(arr_round4[i][0][0])+': 결승 진출확률R: '+str(arr_round4[i][1][0])+': 결승 진출확률avg: '+str((arr_round4[i][0][0]+arr_round4[i][1][0])/2)+': 3,4위전 진출 확률L: '+str(arr_round4[i][0][1])+': 3,4위전 진출 확률R: '+str(arr_round4[i][1][1])+': 3,4위전 진출 확률avg: '+str((arr_round4[i][0][1]+arr_round4[i][1][1])/2))
  sum_winl_dup += arr_round4[i][0][0]
  sum_winr_dup += arr_round4[i][0][1]
  sum_losl_dup += arr_round4[i][1][0]
  sum_losr_dup += arr_round4[i][1][1]
  if((i+1)%32 == 0):
    print('---------------------------------------------------')
    print('조 평균 결승 진출 확률L :\t',sum_winl_dup,'\t조 평균 결승 진출 확률R :\t',sum_winr_dup,'\t조 평균 결승 진출 확률avg :\t',(sum_winl_dup+sum_winr_dup)/2,'\t조 평균 3,4위전 진출 확률L :\t',sum_losl_dup,'\t조 평균 3,4위전 진출 확률R :\t',sum_losr_dup,'\t조 평균 3,4위전 진출 확률avg :\t',(sum_losl_dup+sum_losr_dup)/2)
    print(' ')
    sum_winl_dup = 0;
    sum_winr_dup = 0;
    sum_losl_dup = 0;
    sum_losr_dup = 0;
  
arr_rank = [[0 for j in range(2)] for i in range(32)] #결승 팀별 승패 확률 -> 1등 2등 3등 4등 확률

#여기서는 rank[n][0] / [n][1]만 판명 -> 결승
for cut_groupnum in range(32):
#     print(str(cut_match*8+cut_groupnum))
  if(cut_match%2 == 0): # A C E G
    for winlos in range(2):
      for opp in range(32):
        if(cut_groupnum != opp):
          arr_rank[cut_groupnum][0] += arr_round4[cut_groupnum][winlos][1] * arr_round4[opp][1-winlos][1] * arr_predict[int((cut_groupnum)%8)][int((cut_groupnum)/8)][int((opp)%8)][int((opp)/8)]
          arr_rank[cut_groupnum][1] += arr_round4[cut_groupnum][winlos][1] * arr_round4[opp][1-winlos][1] * (1-arr_predict[int((cut_groupnum)%8)][int((cut_groupnum)/8)][int((opp)%8)][int((opp)/8)])
  else:
    for winlos in range(2):
      for opp in range(32):
        if(cut_groupnum != opp):
          arr_rank[cut_groupnum][0] += arr_round4[cut_groupnum][winlos][1] * arr_round4[opp][1-winlos][1] * arr_predict[int((cut_groupnum)%8)][int((cut_groupnum)/8)][int((opp)%8)][int((opp)/8)]
          arr_rank[cut_groupnum][1] += arr_round4[cut_groupnum][winlos][1] * arr_round4[opp][1-winlos][1] * (1-arr_predict[int((cut_groupnum)%8)][int((cut_groupnum)/8)][int((opp)%8)][int((opp)/8)])



sum_ran_fir = 0;
sum_ran_sec = 0;
for i in range(32):
  sum_ran_fir += arr_rank[i][0]
  sum_ran_sec += arr_rank[i][1]

sum_fir_dup = 0
sum_sec_dup = 0
for i in range(32):
  arr_rank[i][0] /= sum_ran_fir
  arr_rank[i][1] /= sum_ran_sec
  print(arr_country[i]+': 3등 확률: '+str(arr_rank[i][0])+' / 4등 확률: '+str(arr_rank[i][1]))
  sum_fir_dup += arr_rank[i][0]
  sum_sec_dup += arr_rank[i][1]
  if((i+1)%32 == 0):
    print('---------------------------------------------------')
    print('조 평균 3등 확률 :\t',sum_fir_dup,'\t조 평균 4등 확률 :\t',sum_sec_dup)
    print(' ')
    sum_fir_dup = 0
    sum_sec_dup = 0


arr_final = [[0 for j in range(2)] for i in range(32)] #결승 팀별 승패 확률 -> 1등 2등 3등 4등 확률

#여기서는 rank[n][0] / [n][1]만 판명 -> 결승
for cut_groupnum in range(32):
#     print(str(cut_match*8+cut_groupnum))
  if(cut_match%2 == 0): # A C E G
    for winlos in range(2):
      for opp in range(32):
        if(cut_groupnum != opp):
          arr_final[cut_groupnum][0] += arr_round4[cut_groupnum][winlos][0] * arr_round4[opp][1-winlos][0] * arr_predict[int((cut_groupnum)%8)][int((cut_groupnum)/8)][int((opp)%8)][int((opp)/8)]
          arr_final[cut_groupnum][1] += arr_round4[cut_groupnum][winlos][0] * arr_round4[opp][1-winlos][0] * (1-arr_predict[int((cut_groupnum)%8)][int((cut_groupnum)/8)][int((opp)%8)][int((opp)/8)])
  else:
    for winlos in range(2):
      for opp in range(32):
        if(cut_groupnum != opp):
          arr_final[cut_groupnum][0] += arr_round4[cut_groupnum][winlos][0] * arr_round4[opp][1-winlos][0] * arr_predict[int((cut_groupnum)%8)][int((cut_groupnum)/8)][int((opp)%8)][int((opp)/8)]
          arr_final[cut_groupnum][1] += arr_round4[cut_groupnum][winlos][0] * arr_round4[opp][1-winlos][0] * (1-arr_predict[int((cut_groupnum)%8)][int((cut_groupnum)/8)][int((opp)%8)][int((opp)/8)])

sum_fin_fir = 0;
sum_fin_sec = 0;
for i in range(32):
  sum_fin_fir += arr_final[i][0]
  sum_fin_sec += arr_final[i][1]

sum_fir_dup = 0
sum_sec_dup = 0
for i in range(32):
  arr_final[i][0] /= sum_fin_fir
  arr_final[i][1] /= sum_fin_sec
  print(arr_country[i]+': 우승 확률: '+str(arr_final[i][0])+' / 준우승 확률: '+str(arr_final[i][1]))
  sum_fir_dup += arr_final[i][0]
  sum_sec_dup += arr_final[i][1]
  if((i+1)%32 == 0):
    print('---------------------------------------------------')
    print('조 평균 우승 확률 :\t',sum_fir_dup,'\t조 평균 준우승 확률 :\t',sum_sec_dup)
    print(' ')
    sum_fir_dup = 0
    sum_sec_dup = 0


out_result = [[0 for country in range(59)] for data in range(33)]
out_result[0][0] = 'country_number'
out_result[0][1] = 'country_name'
for i in range(32):
  out_result[0][i+2] = 'vs '+ arr_country[i]
out_result[0][34] = '1_in_group' 
out_result[0][35] = '2_in_group'
out_result[0][36] = 'go_to_r16' #[33]+[34]

out_result[0][37] = 'go_to_r8_in_left'
out_result[0][38] = 'go_to_r8_in_right'
out_result[0][39] = 'go_to_r8' #36+37
out_result[0][40] = 'fail_in_r16_left' 
out_result[0][41] = 'fail_in_r16_right'
out_result[0][42] = 'fail_in_r16' # 39 + 40

out_result[0][43] = 'go_to_r4_in_left'
out_result[0][44] = 'go_to_r4_in_right'
out_result[0][45] = 'go_to_r4' #42+43
out_result[0][46] = 'fail_in_r8_left' 
out_result[0][47] = 'fail_in_r8_right'
out_result[0][48] = 'fail_in_r8' # 45 + 46

out_result[0][49] = 'go_to_final_in_left'
out_result[0][50] = 'go_to_final_in_right'
out_result[0][51] = 'go_to_final' #48+49
out_result[0][52] = 'go_to_rank_in_left'
out_result[0][53] = 'go_to_rank_in_right'
out_result[0][54] = 'go_to_rank' #51+52

out_result[0][55] = '3_in_rank'
out_result[0][56] = '4_in_rank'
out_result[0][57] = '1_in_final'
out_result[0][58] = '2_in_final'



for i in range(32):
  out_result[i+1][0] = i+1
  out_result[i+1][1] = arr_country[i]



sum_group = [0 for j in range(32)]
for i in range(32):
  for j in range(32):
    out_result[i+1][j+2] = arr_predict[int(i/4)][i%4][int(j/4)][j%4]

sum_group_win = [0 for i in range(32)]
for i in range(32):
  for j in range(2):
    out_result[i+1][j+34] = arr_groupwin[i][j]
    sum_group_win[i] += arr_groupwin[i][j]

for i in range(32):
  out_result[i+1][36] = sum_group_win[i]
  



sum_r16_win = [0 for i in range(32)]
for i in range(32):
  for k in range(2):
    out_result[i+1][k+37] = arr_round16[i][k][0]
    sum_r16_win[i] += arr_round16[i][k][0]
for i in range(32):
  out_result[i+1][39] = sum_r16_win[i]

sum_r16_los = [0 for i in range(32)]
for i in range(32):
  for k in range(2):
    out_result[i+1][k+40] = arr_round16[i][k][1]
    sum_r16_los[i] += arr_round16[i][k][1]
for i in range(32):
  out_result[i+1][42] = sum_r16_los[i]



sum_r8_win = [0 for i in range(32)]
for i in range(32):
  for k in range(2):
    out_result[i+1][k+43] = arr_round8[i][k][0]
    sum_r8_win[i] += arr_round8[i][k][0]
for i in range(32):
  out_result[i+1][45] = sum_r8_win[i]

sum_r8_los = [0 for i in range(32)]
for i in range(32):
  for k in range(2):
    out_result[i+1][k+46] = arr_round8[i][k][1]
    sum_r8_los[i] += arr_round8[i][k][1]
for i in range(32):
  out_result[i+1][48] = sum_r8_los[i]


sum_r4_win = [0 for i in range(32)]
for i in range(32):
  for k in range(2):
    out_result[i+1][k+49] = arr_round4[i][k][0]
    sum_r4_win[i] += arr_round4[i][k][0]
for i in range(32):
  out_result[i+1][51] = sum_r4_win[i]

sum_r4_los = [0 for i in range(32)]
for i in range(32):
  for k in range(2):
    out_result[i+1][k+52] = arr_round4[i][k][1]
    sum_r4_los[i] += arr_round4[i][k][1]
for i in range(32):
  out_result[i+1][54] = sum_r4_los[i]


for i in range(32):
    out_result[i+1][55] = arr_rank[i][0]

for i in range(32):
    out_result[i+1][56] = arr_rank[i][1]

for i in range(32):
    out_result[i+1][57] = arr_final[i][0]

for i in range(32):
    out_result[i+1][58] = arr_final[i][1]

#print(out_result)

out_result_csv = pd.DataFrame(out_result)
from datetime import datetime

now = datetime.now()

out_result_csv.to_csv('/content/drive/MyDrive/soccer_data/result/logistic_regression_result_'+now.strftime('%y_%m_%d_%H_%M_%S')+'.csv', header=False, index=False)

Mounted at /content/drive
 
Qatar0 vs. Qatar0 / new: 0.5 / old home: 0.45895708966941834 / old away: 0.5410429103305816 /  home wei265.32000000000005 / away wei: 265.32000000000005
Qatar0 vs. Ecuador1 / new: 0.4365736891795543 / old home: 0.4365736891795543 / old away: 0.3025891810481385 /  home wei265.32000000000005 / away wei: 330.09999999999997
Qatar0 vs. Senegal2 / new: 0.2852672624167402 / old home: 0.2852672624167402 / old away: 0.2969615763607121 /  home wei265.32000000000005 / away wei: 569.3100000000001
Qatar0 vs. Netherlands3 / new: 0.193901540826852 / old home: 0.193901540826852 / old away: 0.2979221911040014 /  home wei265.32000000000005 / away wei: 831.42
Qatar0 vs. England4 / new: 0.1995809712507348 / old home: 0.1995809712507348 / old away: 0.3129102052083196 /  home wei265.32000000000005 / away wei: 754.09
Qatar0 vs. Iran5 / new: 0.319442901051321 / old home: 0.319442901051321 / old away: 0.30479488126851717 /  home wei265.32000000000005 / away wei: 462.33000000000004
Q